In [1]:
import math
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.optimize import leastsq
from sklearn.linear_model import LinearRegression
from scipy import sparse
import numpy as np

## 一.品牌匹配

In [5]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.drop(['body','energy'],axis=1)
car_autohome_all = car_autohome_all.loc[:, ['brand_name','brand_slug','model_name','model_slug','detail_name','detail_slug','online_year','price_bn']]
car_autohome_all = car_autohome_all.sort_values(by=['brand_name','model_name','online_year','price_bn'])

# 汽车之家品牌
car_autohome_brand = car_autohome_all.loc[:, ['brand_name','brand_slug']].drop_duplicates(['brand_slug']).rename(columns={'brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
print('汽车之家品牌数量:', len(car_autohome_brand))

liyang_brand = open_category.loc[(open_category['parent'].isnull()), ['id','name']].rename(columns={'name':'brand_name'}).reset_index(drop=True)
print('力洋品牌数量:', len(liyang_brand))
brand = liyang_brand.merge(car_autohome_brand, how='left', on=['brand_name'])
brand = brand.sort_values(by=['brand_name'])
car_autohome_brand = car_autohome_brand.sort_values(by=['brand_name'])
car_autohome_brand.to_csv('../tmp/train/match_car_autohome_brand.csv', index=False)
brand.to_csv('../tmp/train/final_brand.csv', index=False)
brand.info()
print('请人工匹配未匹配上品牌.')

汽车之家品牌数量: 196
力洋品牌数量: 156
<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 114 to 111
Data columns (total 3 columns):
id                       156 non-null int64
brand_name               156 non-null object
car_autohome_brand_id    112 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.9+ KB
请人工匹配未匹配上品牌.


## 二.车系匹配

In [28]:
brand = pd.read_csv('../tmp/train/final_brand.csv')

open_category = pd.read_csv('../tmp/train/open_category.csv')
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv')
open_category_brand = open_category.loc[(open_category['parent'].isnull()), ['id','slug']].rename(columns={'slug':'parent'}).reset_index(drop=True)
brand = open_category_brand.merge(brand,how='left',on=['id'])
model = open_category.loc[(open_category['parent'].notnull()), ['id','name','slug','parent']].rename(columns={'name':'model_name',
                         'slug':'model_slug','id':'model_id'}).reset_index(drop=True)
model = model.merge(brand, how='left', on=['parent']).rename(columns={'id':'brand_id'})
model = model.loc[:, ['brand_id','model_id','brand_name','model_name','model_slug','car_autohome_brand_id']]
print('力洋车系数量:', len(model))

# 汽车之家车系
car_autohome_model = car_autohome_all.loc[:, ['brand_slug','model_slug','model_name']].drop_duplicates(['model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
print('汽车之家车系数量:', len(car_autohome_model))

model = model.merge(car_autohome_model, how='left', on=['car_autohome_brand_id','model_name'])
model.loc[(model['car_autohome_brand_id'].isnull()), 'car_autohome_brand_id'] = -1
model.loc[(model['car_autohome_brand_id'] == -1), 'car_autohome_model_id'] = -1
model = model.sort_values(by=['brand_name','model_name'])
model.to_csv('../tmp/train/final_model.csv', index=False)

temp = car_autohome_all.loc[:, ['model_slug','brand_name','model_name']].drop_duplicates(['model_slug'])                                                                                                                            
temp.to_csv('../tmp/train/match_car_autohome_model.csv', index=False)
model.info()
# model.head()
# model = model.merge(brand, how='left', on=['parent']).rename(columns={'parent':'brand_slug'})

力洋车系数量: 1765
汽车之家车系数量: 1826
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1765 entries, 890 to 539
Data columns (total 7 columns):
brand_id                 1765 non-null int64
model_id                 1765 non-null int64
brand_name               1765 non-null object
model_name               1765 non-null object
model_slug               1765 non-null object
car_autohome_brand_id    1765 non-null float64
car_autohome_model_id    608 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 110.3+ KB


In [47]:
def process_model_name(df):
    text = df['model_name']
    text = text.replace(df['brand_name'],'')
    text = text.replace(' ', '')
    text = text.lower()
    return text

model = pd.read_csv('../tmp/train/final_model.csv')
part1 = model.loc[(model['car_autohome_model_id'].notnull()), :].reset_index(drop=True)
part2 = model.loc[(model['car_autohome_model_id'].isnull()), :].reset_index(drop=True)
part2['model_name'] = part2.apply(process_model_name, axis=1)
part2 = part2.drop(['car_autohome_model_id'], axis=1)
car_autohome_model = car_autohome_all.loc[:, ['brand_slug','brand_name','model_slug','model_name']].drop_duplicates(['model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
car_autohome_model['model_name'] = car_autohome_model.apply(process_model_name, axis=1)
car_autohome_model = car_autohome_model.drop(['brand_name'], axis=1)
part2 = part2.merge(car_autohome_model, how='left', on=['car_autohome_brand_id','model_name'])
final = part1.append(part2, sort=False)
final = final.sort_values(by=['brand_name','model_name'])
final.to_csv('../tmp/train/final_model.csv', index=False)
part2.head()

,brand_id,model_id,brand_name,model_name,model_slug,car_autohome_brand_id,car_autohome_model_id
0,1851,1881,Pagani,zonda,bseries_964,61,NaN
1,1346,1203,一汽,佳宝,jiabao,110,NaN
2,1346,1295,一汽,夏利2000,xiali2000,110,NaN
3,1346,1291,一汽,夏利a+,xiali-ajia,110,NaN
4,1346,1292,一汽,夏利n3,xiali-n3,110,NaN


## 三.款型匹配

In [141]:
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv').rename(columns={'year':'online_year','global_slug':'model_slug'})
open_model_detail = open_model_detail.drop(['volume','control'],axis=1)
final_brand = pd.read_csv('../tmp/train/final_brand.csv')
final_model = pd.read_csv('../tmp/train/final_model.csv')
final_model = final_model.loc[(final_model['car_autohome_model_id'].notnull())&(final_model['car_autohome_model_id'] != -1), :].reset_index(drop=True)
gpj_match_model = open_model_detail.loc[(open_model_detail['model_slug'].isin(list(set(final_model.model_slug.values)))), :]
gpj_match_model = gpj_match_model.merge(final_model.loc[:,['model_slug','car_autohome_brand_id','car_autohome_model_id']],how='left',on=['model_slug'])
print('匹配上车系的力洋款型:', gpj_match_model.info())

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.drop(['manufacturer','body','energy'], axis=1)
print('汽车之家车系总量:',len(list(set(car_autohome_all.model_slug.values))))

# 汽车之家补充的车系
supplement_model = car_autohome_all.loc[~(car_autohome_all['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = car_autohome_all.loc[(car_autohome_all['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = match_model.rename(columns={'model_slug':'car_autohome_model_id','detail_slug':'car_autohome_detail_id'}).loc[:, ['car_autohome_model_id','car_autohome_detail_id','detail_name','price_bn','online_year']]
print('汽车之家补充的车系量:', len(list(set(supplement_model.model_slug.values))))
print('汽车之家匹配的车系量:', len(list(set(match_model.car_autohome_model_id.values))))

final_detail = gpj_match_model.merge(match_model,how='left',on=['car_autohome_model_id','online_year','price_bn'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34902 entries, 0 to 34901
Data columns (total 25 columns):
id                       34902 non-null int64
source_id                20629 non-null float64
checker_runtime_id       17134 non-null float64
old_dmodel               24210 non-null object
detail_model             34902 non-null object
model_detail_slug        34902 non-null object
price_bn                 34902 non-null float64
cont_vprice              29623 non-null float64
url                      17134 non-null object
model_slug               34902 non-null object
domain                   34902 non-null object
status                   34902 non-null object
online_year              34902 non-null int64
has_param                34902 non-null object
vv                       29389 non-null float64
listed_year              34902 non-null int64
delisted_year            18157 non-null float64
emission_standard        34554 non-null object
volume_extend            34603 non-null ob

/home/ml/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [142]:
def cal_match_rate(df):
    a = []
    a.extend(df['detail_model'].replace(' ',''))
    b = []
    b.extend(df['detail_name'].replace(' ',''))
    return float('%.3f' % (len(set(a).intersection(set(b)))/len(set(a))))
    
dup_details = final_detail.groupby(['id'])['model_detail_slug'].count().reset_index()
dup_details = dup_details.loc[(dup_details['model_detail_slug'] > 1), :].reset_index(drop=True)
dup_details = final_detail.loc[(final_detail['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)
# dup_details.info()

part1 = final_detail.loc[~(final_detail['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)

result = pd.DataFrame()
for liyang_id in list(set(dup_details.id.values)):
    temp = dup_details.loc[(dup_details['id'] == liyang_id), :].reset_index(drop=True)
    temp['match_rate'] = temp.apply(cal_match_rate, axis=1)
    result = result.append(temp.loc[(temp['match_rate'] == max(list(set(temp.match_rate.values)))), :].reset_index(drop=True), sort=False)
result.reset_index(inplace=True, drop=True)

dup_details = result.groupby(['id'])['model_detail_slug'].count().reset_index()
dup_details = dup_details.loc[(dup_details['model_detail_slug'] > 1), :].reset_index(drop=True)
dup_details = result.loc[(result['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)

part2 = result.loc[~(result['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)
match_details = part1.append(part2, sort=False)
match_details = match_details.drop(['match_rate'], axis=1)
match_details = match_details.loc[(match_details['car_autohome_detail_id'].notnull()), :]

## 四.品牌车系款型组装

In [143]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
gpj_brand = open_category.loc[(open_category['parent'].isnull()), :].reset_index(drop=True)
gpj_model = open_category.loc[(open_category['parent'].notnull()), :].reset_index(drop=True)
gpj_detail = pd.read_csv('../tmp/train/open_model_detail.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')

final_brand = pd.read_csv('../tmp/train/final_brand.csv')
final_model = pd.read_csv('../tmp/train/final_model.csv')

In [145]:
# 补充品牌
def process_brand(df):
    return 'car_autohome_'+str(df['slug'])

liyang_brand = final_brand.loc[(final_brand['car_autohome_brand_id'].notnull()), :].reset_index(drop=True)
temp = car_autohome_all.loc[(car_autohome_all['brand_slug'].isin(list(set(liyang_brand.car_autohome_brand_id.values)))), :]
liyang_brand = liyang_brand.loc[(liyang_brand['car_autohome_brand_id'].isin(list(set(temp.brand_slug.values)))),:]
supplement_brand = list(set(car_autohome_all.brand_slug.values).difference(set(temp.brand_slug.values)))
supplement_brand = car_autohome_all.loc[(car_autohome_all['brand_slug'].isin(supplement_brand)), ['brand_slug','brand_name']]
supplement_brand = supplement_brand.drop_duplicates(['brand_slug','brand_name']).reset_index(drop=True)
supplement_brand = supplement_brand.rename(columns={'brand_name':'name','brand_slug':'slug'})
supplement_brand['slug'] = supplement_brand.apply(process_brand, axis=1)

gpj_brand = gpj_brand.loc[(gpj_brand['id'].isin(list(set(liyang_brand.id.values)))), :].reset_index(drop=True)
gpj_brand = gpj_brand.append(supplement_brand, sort=False)
gpj_brand.to_csv('../tmp/train/combine_brand.csv', index=False)
print('补充品牌数量:', len(supplement_brand))
print('力洋匹配上品牌数量:', len(liyang_brand))
print('汽车之家品牌数量:', len(list(set(car_autohome_all.brand_slug.values))))

补充品牌数量: 52
力洋匹配上品牌数量: 144
汽车之家品牌数量: 196


In [95]:
# 补充车系
print('补充车系数量:', len(list(set(supplement_model.model_slug.values))))
print('力洋匹配上车系数量:', len(list(set(match_model.car_autohome_model_id.values))))
print('汽车之家车系数量:', len(list(set(car_autohome_all.model_slug.values))))

补充车系数量: 313
力洋匹配上车系数量: 1513
汽车之家车系数量: 1826


In [146]:
gpj_model.head()

,id,source_id,name,alias,slug,url,parent,checker_runtime_id,keywords,classified,...,thumbnail,pinyin,status,attribute,units,popular,on_sale,score,normalized_name,brand_area
0,149,6.0,奥迪A5,NaN,audi-a5,http://www.ganji.com/audi-a5/,audi,7995.0,",aodiA5(jinkou),奥迪A5(进口),奥迪,audi,进口奥迪",中型车,...,6078529378d4aa18701ae82f09d0bbd01c97f8ce.jpg?i...,aodiA5(jinkou),Y,进口,22246,B,1,16491,A5,NaN
1,150,6.0,奥迪A6,NaN,audi-a6,http://www.ganji.com/audi-a6/,audi,7996.0,",aodiA6,奥迪A6,奥迪,audi,一汽奥迪",中大型车,...,55d9494592eea22a700c1191ae789d47.jpg?imageView...,aodiA6,Y,合资,21000,B,0,508,A6,NaN
2,152,6.0,奥迪Q7,NaN,audi-q7,http://www.ganji.com/audi-q7/,audi,7998.0,",aodiQ7(jinkou),奥迪Q7(进口),奥迪,audi,进口奥迪",中大型SUV,...,3ec873db550e2e91cc9c6537b482f95c6e6a8faf.jpg?i...,aodiQ7(jinkou),Y,进口,27183,B,1,24348,Q7,NaN
3,153,6.0,奥迪R8,NaN,audi-r8,http://www.ganji.com/audi-r8/,audi,7999.0,",aodiR8(jinkou),奥迪R8(进口),奥迪,audi,进口奥迪",跑车,...,01d68526eedd50a17d2fdedac5f2b7fefe8c8633.jpg?i...,aodiR8(jinkou),Y,进口,2505,B,1,16447,R8,NaN
4,154,6.0,奥迪S5,NaN,audi-s5,http://www.ganji.com/audi-s5/,audi,8000.0,",aodiS5(jinkou),奥迪S5(进口),奥迪,audi,进口奥迪",中型车,...,92e924271a067d4a821d76ed9beb4c2db843241b.jpg?i...,aodiS5(jinkou),Y,进口,2343,B,1,3004,S5,NaN


In [99]:
# 补充款型
supplement_detail = car_autohome_all.loc[~(car_autohome_all['detail_slug'].isin(list(set(match_details.car_autohome_detail_id.values)))), :]
print('补充款型数量:', len(list(set(supplement_detail.detail_slug.values))))
print('力洋匹配上款型数量:', len(list(set(match_details.car_autohome_detail_id.values))))
print('汽车之家款型数量:', len(list(set(car_autohome_all.detail_slug.values))))

补充款型数量: 8621
力洋匹配上款型数量: 22834
汽车之家款型数量: 31455


In [100]:
gpj_brand.head()

,id,source_id,name,alias,slug,url,parent,checker_runtime_id,keywords,classified,...,thumbnail,pinyin,status,attribute,units,popular,on_sale,score,normalized_name,brand_area
0,1,5.0,奥迪,NaN,audi,http://www.ganji.com/audi/,NaN,7847.0,",aodi,奥迪,audi",NaN,...,531912ec472bd0568ddc05ca43eb71ac.png?imageView...,aodi,Y,NaN,0,B,1,607387,NaN,德系
1,2,5.0,阿尔法罗密欧,NaN,alfaromeo,http://www.ganji.com/alfaromeo/,NaN,7848.0,",aerfaluomiou,阿尔法罗米欧,alfaromeo",NaN,...,590ae329607ce6a295c3334210514576.png?imageView...,aerfaluomiou,Y,NaN,0,B,0,0,NaN,欧系
2,3,5.0,阿斯顿马丁,NaN,astonmartin,http://www.ganji.com/astonmartin/,NaN,7849.0,",asidunmading,阿斯顿马丁,astonmartin",NaN,...,61b76563144100050d0113f33dfde492.png?imageView...,asidunmading,Y,NaN,0,B,1,0,NaN,欧系
3,6,5.0,奔驰,NaN,benz,http://www.ganji.com/benz/,NaN,7852.0,",benchi,奔驰,benz",NaN,...,9af2620681581d6eec47907b8823855d.png?imageView...,benchi,Y,NaN,0,B,1,404081,NaN,德系
4,7,5.0,别克,NaN,buick,http://www.ganji.com/buick/,NaN,7853.0,",bieke,别克,buick",NaN,...,c41d3f6bac0a5b4e042ca4235405730c.png?imageView...,bieke,Y,NaN,0,B,1,376722,NaN,美系


In [79]:
car_autohome_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31455 entries, 0 to 31454
Data columns (total 11 columns):
detail_slug     31455 non-null int64
brand_name      31455 non-null object
brand_slug      31455 non-null int64
model_name      31455 non-null object
model_slug      31455 non-null int64
detail_name     31455 non-null object
manufacturer    31455 non-null object
price_bn        31455 non-null float64
body            31455 non-null object
energy          31455 non-null object
online_year     31455 non-null int64
dtypes: float64(1), int64(4), object(6)
memory usage: 2.6+ MB


In [82]:
match_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854 entries, 0 to 27853
Data columns (total 5 columns):
car_autohome_model_id     27854 non-null int64
car_autohome_detail_id    27854 non-null int64
detail_name               27854 non-null object
price_bn                  27854 non-null float64
online_year               27854 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.1+ MB
